# 1. 모델 학습

1-1. 데이터 로드

In [ ]:
#모듈 불러오기
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#데이터 불러오기 및 확인
df = pd.read_csv("dataset_used_car/used_car.csv")
df #데이터 출력

In [ ]:
print(df.shape) #데이터 크기
print(df.info()) #범주형 변수, 수치형 변수 확인

#범주형 데이터 확인, 범주형 별수별 카테고리 출력
print(df["Location"].unique())
print(df["Fuel_Type"].unique())
print(df["Transmission"].unique())
print(df["Owner_Type"].unique())

1-2. 데이터셋 전처리

In [ ]:
#결측치 검출
df.isnull().sum() #열별 결측치 행 수 파악
df.drop(["New_Price"],axis=1, inplace=True) #결측치가 많아 분석에 무의미한 New_Price 열 제거
df.dropna(inplace=True) #열별 결측치 행 수 파악, 결측치 없음 확인
df.isnull().sum()

In [ ]:
df.shape #데이터셋 크기 확인  5872 x 12
df = df.reset_index(drop=True) #인덱스 리셋
df.tail() #데이터셋의 마지막 위치 인덱스 확인

In [ ]:
#범주형 데이터에서 수치 분리, 각 열을 공백으로 분리하고 첫번째 텍스트만 추출
df["Mileage_kmpl"] = df["Mileage"].str.split(" ").str[0]
df["Engine_cc"] = df["Engine"].str.split(" ").str[0]
df["Power_bhp"] = df["Power"].str.split(" ").str[0]
df #결과 확인

In [ ]:
#분리한 열은 텍스트이므로 실수형으로 변환
df["Mileage_kmpl"] = df["Mileage_kmpl"].astype(float)
df["Engine_cc"] = df["Engine_cc"].astype(float)
df["Power_bhp"] = df["Power_bhp"].astype(float)

In [ ]:
#불필요 열 삭제
df.drop(["Name"], axis=1, inplace=True)
df.drop(["Mileage"], axis=1, inplace=True)
df.drop(["Engine"], axis=1, inplace=True)
df.drop(["Power"], axis=1, inplace=True)
df.drop(["Location"], axis=1, inplace=True)	

#범주형 데이터 확인
df.info() 	
var_list = ["Fuel_Type", "Transmission", "Owner_Type"]
for var in var_list:
    print(var)
    print(df[var].value_counts())	

#수치형 데이터 정보 요약
df.describe()

In [ ]:
#시각화
df["Price"].plot.hist(bins=100,color="lightblue", edgecolor="red") #Price 분포
df.boxplot(column=["Price"], by="Fuel_Type") #연료별 가격대

In [ ]:
#더미변수 생성, 범주형 데이터를 예측모델에 활용가능하도록 수치형으로 변환
#Fuel_Type 등 범주형 변수를 카테고리별로 분리한 후, 데이터가 있는 열을 1로 표시
train = pd.get_dummies(df)
train #5872 X 17 크기의 데이터 확인

In [ ]:
#독립변수 평균 및 표준편차 계산
stats = train.describe() #데이터 요약
stats.pop("Price") #종속변수 제거
stats = stats.transpose() #행-열 전치
stats.to_csv("used_stats.csv") #모델 활용을 위해 stats를 csv로 저장
stats #데이터 확인

2-3. 모델링 및 평가

In [ ]:
#데이터 분할
x_train = train.sample(frac=0.8, random_state=0) #데이터셋 분리 train 80%
x_test = train.drop(x_train.index) #test 데이터셋 생성
y_train = x_train.pop("Price") #Price를 x_train에서 제거 후 y_train에 저장
y_test = x_test.pop("Price") #Price를 x_test에서 제거 후 y_test에 저장

#데이터셋 표준화, 모델의 정확도 향상을 위해 극단값에 의한 영향을 제거
def norm(x):
  return (x - stats["mean"]) / stats["std"] #x를 한행씩 읽어와서 표준화 후 리턴
normed_train = norm(x_train) #x_train 표준화
normed_test = norm(x_test) #y_train 표준화 

normed_train.shape, normed_test.shape #데이터셋 크기 확인

In [ ]:
#네트워크 구조 정의
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation="relu", input_shape=[len(x_train.keys())]),
  tf.keras.layers.Dense(128, activation="relu"), 
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dense(1)])

model.summary() #네트워크 구조 출력

In [ ]:
#모델 컴파일
#컴파일시 손실함수로 mse, 최적화 함수로 Adam, 평가지표로 mae, mse 사용
optimizer = tf.keras.optimizers.Adam(0.001)
#optimizer = tf.keras.optimizers.RMSprop(0.001) #변경해서 학습해볼것
model.compile(loss="mse", optimizer=optimizer, metrics=["mae", "mse"]) 

#학습 및 평가
#train 데이터 중 25%를 validation 데이터로 나눠 학습, val_loss가 20 epochs 동안 향상이 안될때 학습 종료
history= model.fit(normed_train, y_train, epochs=500, batch_size=64, validation_split=0.25,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=20, monitor="val_loss")])

model.evaluate(normed_test, y_test, verbose=0)	#test 데이터셋으로 모델 평가

In [ ]:
#히스토리 출력
hist = pd.DataFrame(history.history)
hist["epoch"] = history.epoch
hist.tail()

In [ ]:
#train, validation 데이터셋의 mae, mse 그래프 출력
hist = pd.DataFrame(history.history) #history.history에 저장된 값을 데이터프레임으로 변환
hist["epoch"] = history.epoch #hist 데이터프레임에 epoch열 추가

plt.figure(figsize=(12, 4))
plt.subplot(1,2,1) #MAE 그래프 출력
plt.xlabel("Epoch")
plt.ylabel("Mean Abs Error")
plt.plot(hist["epoch"], hist["mae"], label="Train Error")
plt.plot(hist["epoch"], hist["val_mae"], label = "Val Error")
plt.legend()

plt.subplot(1,2,2) #MSE 그래프 출력
plt.xlabel("Epoch")
plt.ylabel("Mean Square Error")
plt.plot(hist["epoch"], hist["mse"], label="Train Error")
plt.plot(hist["epoch"], hist["val_mse"], label = "Val Error")
plt.legend()
plt.show()

In [ ]:
#테스트 데이터로 원래 데이터와 예측결과 비교
pred = model.predict(normed_test)
plt.scatter(y_test,pred, marker=".")
plt.xlabel("Price")
plt.ylabel("Predict")
plt.show()

#결정계수(r squared) 산출
y = y_test.to_numpy()
ybar = np.sum(y) / len(y)
ssr = np.sum((pred - ybar) ** 2)
sst =  np.sum((y - ybar) ** 2)
r_squared = ssr / sst
print(round(r_squared * 100,2), "%")

#케라스 모델로 저장
save_path = "used_model.h5"
model.save(save_path, include_optimizer=True)

# 2. 학습된 모델로 추론(test)

In [ ]:
#모듈 불러오기
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#데이터 & 모델 불러오기
df_test = pd.read_csv("dataset_used_car/used_car_test.csv")
stats = pd.read_csv("used_stats.csv", index_col=0) #첫번째 열을 인덱스로 지정
model = tf.keras.models.load_model("used_model.h5")

#결측치 검출
df_test.isnull().sum()

#예측용 df 생성
df_pred = df_test.copy()

In [ ]:
#범주형 데이터에서 수치 분리
df_pred["Mileage_kmpl"] = df_pred["Mileage"].str.split(" ").str[0]
df_pred["Engine_cc"] = df_pred["Engine"].str.split(" ").str[0]
df_pred["Power_bhp"] = df_pred["Power"].str.split(" ").str[0]

#실수형으로 변환
df_pred["Mileage_kmpl"] = df_pred["Mileage_kmpl"].astype(float)
df_pred["Engine_cc"] = df_pred["Engine_cc"].astype(float)
df_pred["Power_bhp"] = df_pred["Power_bhp"].astype(float)

#불필요 행 삭제
df_pred.drop(["Name"], axis=1, inplace=True)
df_pred.drop(["Mileage"], axis=1, inplace=True)
df_pred.drop(["Engine"], axis=1, inplace=True)
df_pred.drop(["Power"], axis=1, inplace=True)
df_pred.drop(["Location"], axis=1, inplace=True)

#더미변수 생성
df_pred = pd.get_dummies(df_pred)

In [ ]:
#데이터셋 표준화
def norm(x):
  return (x - stats["mean"]) / stats["std"] #x를 한행씩 읽어와서 표준화 후 리턴
normed = norm(df_pred) #데이터셋 표준화

#예측
pred = model.predict(normed)

#원본 데이터셋에 예측열 생성
df_test["Price_predict"] = pred

#원본 데이터와 예측값을 csv 파일로 저장
df_test.to_csv("used_car_predict.csv", index=False)